In [1]:
import sys
print(sys.executable)


C:\Users\mahnoor\anaconda3\python.exe


# Code

In [7]:
import mysql.connector

# Connect to the MySQL server
conn = mysql.connector.connect(
    host="localhost",       # Your MySQL server host
    user="root",            # Your MySQL username
    password="",            # Your MySQL password
    database="viz_optilytics"  # The database you want to use
)

cursor = conn.cursor()

# SQL command to create the table
create_table_query = """
CREATE TABLE vehicles (
    id INT AUTO_INCREMENT PRIMARY KEY,
    color VARCHAR(50),
    type VARCHAR(50),
    model VARCHAR(50),
    timestamp DATETIME
);
"""

try:
    cursor.execute(create_table_query)
    print("Table 'vehicles' created successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

# Close the connection
cursor.close()
conn.close()


Table 'vehicles' created successfully.


In [2]:
import mysql.connector

# Connect to the MySQL server
conn = mysql.connector.connect(
    host="localhost",       # Your MySQL server host
    user="root",            # Your MySQL username
    password="",            # Your MySQL password
    database="viz_optilytics"  # The database you want to use
)

cursor = conn.cursor()

# SQL command to create the table
create_table_query = """
CREATE TABLE people (
    id INT AUTO_INCREMENT PRIMARY KEY,
    people_count INT,
    timestamp DATETIME
);
"""

try:
    cursor.execute(create_table_query)
    print("Table 'people' created successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

# Close the connection
cursor.close()
conn.close()


Table 'people' created successfully.


In [1]:
import sys
from PyQt5 import QtWidgets
from PyQt5.QtWidgets import QDialog, QApplication, QMessageBox
from PyQt5.uic import loadUi
import mysql.connector as con
import os
import cv2
import torch
from PyQt5.QtWidgets import QMainWindow, QWidget, QVBoxLayout, QPushButton, QLabel, QGridLayout
from PyQt5.QtCore import Qt, QTimer, pyqtSignal
from PyQt5.QtGui import QImage, QPixmap, QFont
import time
import pandas as pd
import numpy as np

######################################################################################
# Load color reference CSV and create color lookup
index = ["color", "color_name", "hex", "R", "G", "B"]
csv = pd.read_csv('colors.csv', names=index, header=None)

def get_color_name(bgr):
    b, g, r = bgr
    minimum = 10000
    cname = "Unknown"
    for i in range(len(csv)):
        d = abs(r - int(csv.loc[i, "R"])) + abs(g - int(csv.loc[i, "G"])) + abs(b - int(csv.loc[i, "B"]))
        if d <= minimum:
            minimum = d
            cname = csv.loc[i, "color_name"]
    return cname

# Function to store vehicle information in the database
def store_vehicle_info(color, vehicle_type, model, timestamp):
    conn = con.connect(
        host="localhost",
        user="root",
        password="",
        database="viz_optilytics"
    )
    cursor = conn.cursor()

    sql = "INSERT INTO vehicles (color, type, model, timestamp) VALUES (%s, %s, %s, %s)"
    values = (color, vehicle_type, model, timestamp)

    cursor.execute(sql, values)
    conn.commit()
    cursor.close()
    conn.close()

# Function to extract color from bounding box
def extract_color(frame, bbox):
    x1, y1, x2, y2 = bbox
    roi = frame[y1:y2, x1:x2]
    average_color = np.mean(roi, axis=(0, 1))
    color_name = get_color_name(average_color)
    return color_name
#######################################################################################

class LoginApp(QDialog):
    login_successful = pyqtSignal()

    def __init__(self):
        super(LoginApp, self).__init__()
        loadUi("login_form.ui", self)
        self.b1.clicked.connect(self.login)
        self.b2.clicked.connect(self.show_reg)

    def login(self):
        un = self.tb1.text()
        pw = self.tb2.text()
        db = con.connect(host="localhost", user="root", password="", database="viz_optilytics")
        cursor = db.cursor()
        query = "SELECT * FROM userlist WHERE username=%s AND password=%s"
        cursor.execute(query, (un, pw))
        result = cursor.fetchone()
        self.tb1.setText("")
        self.tb2.setText("")
        if result:
            QMessageBox.information(self, "Login Output", "You logged in successfully :D")
            self.login_successful.emit()
        else:
            QMessageBox.information(self, "Login Output", "Invalid user... Register as new user")

    def show_reg(self):
        widget.setCurrentIndex(1)


class RegApp(QDialog):
    def __init__(self):
        super(RegApp, self).__init__()
        loadUi("register_form.ui", self)
        self.b3.clicked.connect(self.reg)
        self.b4.clicked.connect(self.show_login)
        
    def reg(self):
        em = self.tb3.text()
        un = self.tb4.text()
        pw = self.tb5.text()
        vf = self.tb6.text()
        
        if pw != vf:
            QMessageBox.warning(self, "Registration Error", "Passwords do not match. Please try again.")
            return
        
        db = con.connect(host="localhost", user="root", password="", database="viz_optilytics")
        cursor = db.cursor()
        query = "SELECT * FROM userlist WHERE username=%s"
        cursor.execute(query, (un,))
        result = cursor.fetchone()
        
        if result:
            QMessageBox.information(self, "Login Form", "The user already registered, try another username")
        else:
            query = "INSERT INTO userlist (email, username, password) VALUES (%s, %s, %s)"
            cursor.execute(query, (em, un, pw))
            db.commit()
            QMessageBox.information(self, "Login form", "The user registered successfully, Login now")
            self.tb3.setText("")
            self.tb4.setText("")
            self.tb5.setText("")
            self.tb6.setText("")
            widget.setCurrentIndex(0)
            
    def show_login(self):
        widget.setCurrentIndex(0)


class ObjectDetector:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = torch.hub.load("ultralytics/yolov5", "yolov5s", pretrained=True).to(self.device).eval()
        self.class_names = self.model.module.names if hasattr(self.model, "module") else self.model.names

    def detect_objects(self, frame):
        results = self.model(frame)
        return results.xyxy[0].cpu().numpy()


class VideoWidget(QWidget):
    def __init__(self, parent=None):
        super(VideoWidget, self).__init__(parent)
        self.parent_window = parent
        self.video_label = QLabel()
        self.video_label.setAlignment(Qt.AlignCenter)
        layout = QVBoxLayout()
        layout.addWidget(self.video_label)
        self.setLayout(layout)
        self.capture = cv2.VideoCapture(1)  # Capture camera 1
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(30)  # Update frame every 30 milliseconds
        self.object_detector = None
        self.is_object_detection_started = False
        self.vehicle_positions = {}
        self.static_threshold = 2  # 2 minutes in seconds
        self.alert_generated = False
        

    def update_frame(self):
        #########################################################################
        # Initialize vehicle data dictionary
        # Ensure this is initialized outside if you want to keep state across frames
        if not hasattr(self, 'vehicle_data'):
            self.vehicle_data = {}
            self.person_data = {}
        ###########################################################################

        if self.capture is not None and self.capture.isOpened():
            ret, frame = self.capture.read()
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                if self.is_object_detection_started and self.object_detector:
                    detections = self.object_detector.detect_objects(frame_rgb)
                    current_time = time.time()
                    
                    person_count = 0 
                    
                    for detection in detections:
                        label = self.object_detector.class_names[int(detection[5])]
                        if label in ['car', 'truck', 'bus', 'bike']:  # Detect only vehicles
                            confidence = detection[4]
                            if confidence > 0.5:  # Adjust confidence threshold as needed
                                x1, y1, x2, y2 = map(int, detection[:4])
                                box_id = f"{x1}-{y1}-{x2}-{y2}"

                                if box_id not in self.vehicle_data:
                                    self.vehicle_data[box_id] = {'start_time': current_time, 'last_position': (x1, y1, x2, y2)}
                                    print(f"Initialized start_time for {box_id} at {self.vehicle_data[box_id]['start_time']}")
    
                                # Calculate elapsed time
                                elapsed_time = current_time - self.vehicle_data[box_id]['start_time']
                                print(f"Elapsed time for {box_id}: {elapsed_time:.2f} seconds")

                                if elapsed_time > self.static_threshold:
                                    color = (0, 0, 255)  # Red for static vehicles
                                    self.alert_generated = True
                                else:
                                    color = (0, 255, 0)  # Green for moving vehicles

                                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                                cv2.putText(frame, f"{label}: {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                                # Vehicle data handling
                                if self.vehicle_data[box_id]['last_position'] == (x1, y1, x2, y2):
                                    print("previous and current situation have been matched")
                                    if elapsed_time > 2:
                                        print("storing the details")
                                        # Extract vehicle details and store them
                                        vehicle_color = extract_color(frame, (x1, y1, x2, y2))
                                        vehicle_type = label
                                        vehicle_model = "Unknown Model"  # Placeholder
                                        timestamp = time.strftime('%Y-%m-%d %H:%M:%S')

                                        # Store to database
                                        print(f"Storing data for vehicle: Color={vehicle_color}, Type={vehicle_type}, Model={vehicle_model}, Time={timestamp}")
                                        store_vehicle_info(vehicle_color, vehicle_type, vehicle_model, timestamp)
                                else:
                                    print("positions did not match")

                        elif label == 'person':  # Detect only people
                            print("I am dtecting a person")
                            confidence = detection[4]
                            if confidence > 0.5:  # Adjust confidence threshold as needed
                                x1, y1, x2, y2 = map(int, detection[:4])
                                box_id = f"{x1}-{y1}-{x2}-{y2}"  # Unique ID for the detected person

                                if box_id not in self.person_data:
                                    self.person_data[box_id] = {'start_time': current_time, 'last_position': (x1, y1, x2, y2)}
                                    print(f"Initialized start_time for {box_id} at {self.person_data[box_id]['start_time']}")

                                elapsed_time = current_time - self.person_data[box_id]['start_time']
                                print(f"Elapsed time for {box_id}: {elapsed_time:.2f} seconds")

                                if elapsed_time > 2:  # 2 seconds threshold
                                    person_count += 1
                                    color = (0, 0, 255)  # Red for alert
                                    self.alert_generated = True
                                else:
                                    color = (0, 255, 0)  # Green for moving people

                                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                                cv2.putText(frame, f"{label}: {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                                    
                    # Update status strip based on alert
                    self.parent_window.update_status(self.alert_generated)

                image = QImage(frame, frame.shape[1], frame.shape[0], QImage.Format_RGB888)
                self.video_label.setPixmap(QPixmap.fromImage(image))



    def start_recording(self):
        self.capture = cv2.VideoCapture(0)  # Open camera 0 for recording
        if self.capture.isOpened():
            self.timer.start(30)  # Start timer to update frame
            self.is_recording = True

    def stop_recording(self):
        self.capture.release()
        self.timer.stop()
        self.object_detector = None
        self.capture = cv2.VideoCapture(1)  # Switch to camera 1
        if self.capture.isOpened():
            self.timer.start(30)  # Start timer to update frame

    def start_object_detection(self):
        if self.object_detector is None:
            self.object_detector = ObjectDetector()
            self.is_object_detection_started = True


class StatusStrip(QWidget):
    def __init__(self):
        super().__init__()
        self.status_label = QLabel("Status: No Alerts")
        self.status_label.setAlignment(Qt.AlignCenter)
        self.status_label.setFont(QFont("Arial", 12, QFont.Bold))
        self.setStyleSheet("background-color: green; color: white;")
        layout = QVBoxLayout()
        layout.addWidget(self.status_label)
        self.setLayout(layout)

    def update_status(self, alert):
        if alert:
            self.status_label.setText("Status: Alert Generated")
            self.setStyleSheet("background-color: red; color: white;")
        else:
            self.status_label.setText("Status: No Alerts")
            self.setStyleSheet("background-color: green; color: white;")


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Video Streaming and Object Detection")
        self.setGeometry(100, 100, 1200, 800)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        self.video_widget = VideoWidget(self)

        start_recording_button = QPushButton("Start Recording")
        start_recording_button.clicked.connect(self.start_recording)

        stop_recording_button = QPushButton("Stop Recording")
        stop_recording_button.clicked.connect(self.stop_recording)

        start_detection_button = QPushButton("Start Object Detection")
        start_detection_button.clicked.connect(self.start_detection)

        # Create a grid layout
        layout = QGridLayout()

        # Status strip
        self.status_strip = StatusStrip()
        layout.addWidget(self.status_strip, 0, 0, 1, 4)

        # Add widgets to the grid layout
        layout.addWidget(self.create_section_widget("Camera Feed", self.video_widget), 1, 0, 2, 3)
        layout.addWidget(self.create_section_widget("Detection", start_detection_button), 1, 3)
        layout.addWidget(self.create_section_widget("Controls", start_recording_button, stop_recording_button), 2, 3)
        layout.addWidget(self.create_section_widget("Unauthorized Objects"), 3, 0)
        layout.addWidget(self.create_section_widget("View"), 3, 1)
        layout.addWidget(self.create_section_widget("Alerts"), 3, 2)
        layout.addWidget(self.create_section_widget("Detected"), 3, 3)

        # Set the layout to the central widget
        self.central_widget.setLayout(layout)

    def create_section_widget(self, title, *widgets):
        section_widget = QWidget()
        layout = QVBoxLayout()
        label = QLabel(title)
        label.setAlignment(Qt.AlignCenter)
        label.setFont(QFont("Arial", 14, QFont.Bold))
        layout.addWidget(label)
        for widget in widgets:
            layout.addWidget(widget)
        section_widget.setLayout(layout)
        return section_widget

    def start_recording(self):
        self.video_widget.start_recording()

    def stop_recording(self):
        self.video_widget.stop_recording()

    def start_detection(self):
        self.video_widget.start_object_detection()  # Call VideoWidget's start_object_detection method

    def update_status(self, alert):
        self.status_strip.update_status(alert)
#         self.status_strip.update_status(alert, head_count)


if __name__ == '__main__':
    app = QApplication(sys.argv)
    
    # Login and Registration Widget
    widget = QtWidgets.QStackedWidget()
    loginform = LoginApp()
    registrationform = RegApp()
    
    widget.addWidget(loginform)
    widget.addWidget(registrationform)
    widget.setCurrentIndex(0)  # Show Login form
    widget.setFixedWidth(400)
    widget.setFixedHeight(500)
    widget.show()

    def show_main_window():
        mainWindow = MainWindow()
        mainWindow.show()
        widget.close()  # Close the login/registration window

    loginform.login_successful.connect(show_main_window)
    
    sys.exit(app.exec_())


Using cache found in C:\Users\mahnoor/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-25 Python-3.11.4 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Initialized start_time for 218-86-634-477 at 1725275540.9128764
Elapsed time for 218-86-634-477: 0.00 seconds
Initialized start_time for 219-85-633-475 at 1725275541.2822905
Elapsed time for 219-85-633-475: 0.00 seconds
Initialized start_time for 201-87-639-475 at 1725275541.6453986
Elapsed time for 201-87-639-475: 0.00 seconds
Initialized start_time for 206-91-640-473 at 1725275541.9834776
Elapsed time for 206-91-640-473: 0.00 seconds
Initialized start_time for 202-91-634-473 at 1725275542.3149953
Elapsed time for 202-91-634-473: 0.00 seconds
Initialized start_time for 181-97-634-476 at 1725275542.6690834
Elapsed time for 181-97-634-476: 0.00 seconds
Initialized start_time for 178-117-627-474 at 1725275543.0166001
Elapsed time for 178-117-627-474: 0.00 seconds
Initialized start_time for 181-117-620-474 at 1725275543.3705375
Elapsed time for 181-117-620-474: 0.00 seconds
Initialized start_time for 190-116-620-473 at 1725275543.7180598
Elapsed time for 190-116-620-473: 0.00 seconds
Init

SystemExit: 0